# Tutorial: a simple simulation of alanine dipeptide with MACE using OpenMM-Torch

You can run this tutorial directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm-torch/blob/mace_tutorial/tutorials/openmm-torch-MACE.ipynb)

**Note this notebook should work in both the default and GPU accelerated Colab runtimes**

Covered topics:
 * Installation of the software with [Conda](https://docs.conda.io/)
 * Creation of an NNP (neural network potential) with [MACE](https://github.com/ACEsuit/mace)
 * Integration of [OpenMM](https://openmm.org/) and [PyTorch](https://pytorch.org/) and with [OpenMM-Torch](https://github.com/openmm/openmm-torch)
 * Setup and execution of a simulation

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

## Install software

The [conda-forge](https://conda-forge.org/) channel is used for software.

⚠️ The installation might take up to 10 min!

In [ ]:
#https://github.com/openmm/openmm-torch/issues/88
%env CONDA_OVERRIDE_CUDA=11.2 
!mamba install -c conda-forge -c "conda-forge/label/openmm-torch_rc" \
               -c "conda-forge/label/openmm_rc" \
               openmm=8 openmm-torch=1 openmmtools pytorch=1.12 \
               &> /dev/null # Comment this line to see a log

# install MACE
!pip install -q git+https://github.com/sef43/mace@torchscript_merge 

# get pretrained model
!wget -q https://github.com/jharrymoore/mace-openmm/raw/main/tests/example_data/MACE_SPICE_larger.model


## Prepare a simulation system

For simplicity, the alanine dipeptide system from [OpenMM-Tools](https://openmmtools.readthedocs.io/en/latest/) is used.

In [1]:
import openmmtools

# Get the system of alanine dipeptide
ala2 = openmmtools.testsystems.AlanineDipeptideVacuum(constraints=None)

# Remove MM forces
while ala2.system.getNumForces() > 0:
  ala2.system.removeForce(0)

# The system should not contain any additional force and constrains
assert ala2.system.getNumConstraints() == 0
assert ala2.system.getNumForces() == 0

# Get the list of atomic numbers
atomic_numbers = [atom.element.atomic_number for atom in ala2.topology.atoms()]

# Get positions
positions = ala2.positions

## Create a NNP

A NNP (neural network potential) is created with MACE

In [2]:
import torch
from mace.calculators import MACE_openmm_NNP

device = "cuda" if torch.cuda.is_available() else "cpu"

model_path="MACE_SPICE_larger.model"

nnp = MACE_openmm_NNP(model_path, torch.tensor(positions.tolist()).to(device), atomic_numbers, device=device)

At this point, the potential energy of the system can be computed with the NNP:

In [3]:
# Comute the potential energy
pos = torch.tensor(ala2.positions.tolist()).to(device)
energy_1 = float(nnp(pos))
print(energy_1)

-1302713.6806943913


## Add the NNP to the system

In order to use the NNP in a simulation, it has to loaded with [OpenMM-Torch](https://github.com/openmm/openmm-torch) and added to the system.

In [4]:
from openmmtorch import TorchForce

# Save the NNP to a file and load it with OpenMM-Torch
torch.jit.script(nnp).save('model.pt')
force = TorchForce('model.pt')

# Add the NNP to the system
ala2.system.addForce(force)
assert ala2.system.getNumForces() == 1

## Setup a simulation

Setup a simulation with [OpenMM](https://openmm.org/).

In [5]:
import sys
from openmm import LangevinMiddleIntegrator
from openmm.app import Simulation, StateDataReporter, PDBReporter
from openmm.unit import kelvin, picosecond, femtosecond

# Create an integrator with a time step of 1 fs
temperature = 298.15 * kelvin
frictionCoeff = 1 / picosecond
timeStep = 1 * femtosecond
integrator = LangevinMiddleIntegrator(temperature, frictionCoeff, timeStep)

# Create a simulation and set the initial positions and velocities
simulation = Simulation(ala2.topology, ala2.system, integrator)
simulation.context.setPositions(ala2.positions)

# Configure a reporter to print to the console every 0.1 ps (100 steps)
reporter = StateDataReporter(file=sys.stdout, reportInterval=100, step=True, time=True, potentialEnergy=True, temperature=True, speed=True)
simulation.reporters.append(reporter)
simulation.reporters.append(PDBReporter("output.pdb", 100))

At this point, the potential energy of the system can be computed again:

In [6]:
from openmm.unit import kilojoule_per_mole

# Comute the potential energy
state = simulation.context.getState(getEnergy=True)
energy_2 = state.getPotentialEnergy().value_in_unit(kilojoule_per_mole)
print(energy_2)

# Check if the energy is correct
assert torch.isclose(torch.tensor(energy_1), torch.tensor(energy_2))

-1302713.6806207714


## Run the simulation

Run your first NNP simulation.

⚠️ The simulations are not deterministic! Each time the log will be a bit different.

In [7]:
# Run the simulations for 1 ps (1000 steps)
simulation.step(1000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-1302716.8853482394,51.81304603859144,0
200,0.20000000000000015,-1302714.8697117907,83.10314429857384,0.792
300,0.3000000000000002,-1302701.7569842234,107.27109340114595,0.782
400,0.4000000000000003,-1302694.1482679592,126.0759773132453,0.787
500,0.5000000000000003,-1302695.979146789,131.49732483563452,0.797
600,0.6000000000000004,-1302693.5240542528,140.2359797902038,0.789
700,0.7000000000000005,-1302692.2446522296,194.124386064068,0.785
800,0.8000000000000006,-1302678.9072102618,212.21471257005481,0.791
900,0.9000000000000007,-1302689.6729080956,270.5078155949723,0.796
1000,1.0000000000000007,-1302665.8971019273,196.48169477464165,0.799
